In [1]:
import pandas as pd
df = pd.read_csv('D:/Data/fake-news/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [2]:
df.shape

(20800, 5)

In [4]:
df = df.dropna()
df.shape

(18285, 5)

In [5]:
y = df['label']

In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re

In [7]:
X = df.drop('label', axis=1)
messages = X.copy()

In [8]:
messages.reset_index(inplace=True)

In [9]:
ps = PorterStemmer()
corpus = []

for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [10]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [11]:
from tensorflow.keras.preprocessing.text import one_hot
onehot_repr=[one_hot(words, n=10000)for words in corpus] 
onehot_repr

[[3398, 6079, 466, 2913, 9584, 9975, 3445, 3474, 6490, 6540],
 [1992, 9426, 453, 8856, 4176, 3203, 648],
 [2105, 1466, 1988, 6008],
 [5690, 2383, 1020, 5088, 1981, 889],
 [6473, 4176, 5462, 2900, 6955, 7437, 4176, 5971, 3161, 3976],
 [5381,
  7929,
  6349,
  5459,
  3086,
  6019,
  7626,
  6184,
  1524,
  8505,
  3810,
  7051,
  3525,
  8364,
  648],
 [8642, 4927, 8858, 6682, 2654, 3578, 9936, 299, 2089, 8585, 2473],
 [5355, 4048, 922, 3018, 9401, 8048, 6019, 5414, 2089, 8585, 2473],
 [1500, 8107, 7108, 2470, 1983, 3690, 1166, 5698, 6019, 8452],
 [345, 2856, 9825, 6587, 9493, 8107, 1336, 7709],
 [4126, 7356, 8507, 3373, 8057, 3170, 6564, 8461, 4507, 7469, 7555],
 [5088, 7518, 9584, 3690, 6019, 9401],
 [3631, 1764, 4380, 8596, 7159, 7516, 2449, 6519, 2580],
 [3775, 3635, 2960, 9231, 5856, 5156, 3066, 2089, 8585, 2473],
 [4490, 318, 7246, 9457, 8330, 2089, 8585, 2473],
 [2786, 7506, 8811, 8881, 1105, 884, 4227, 8951, 6846, 7162],
 [8750, 8619, 9426],
 [2266, 8887, 7046, 7857, 6019, 3154,

In [12]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3474 6490 6540]
 [   0    0    0 ... 4176 3203  648]
 [   0    0    0 ... 1466 1988 6008]
 ...
 [   0    0    0 ... 2089 8585 2473]
 [   0    0    0 ... 9738  630 5399]
 [   0    0    0 ... 3464 4610 4684]]


In [14]:
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.models import Sequential
model=Sequential()
model.add(Embedding(10000, 40, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            400000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [16]:
embedded_docs.shape, y.shape

((18285, 20), (18285,))

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [18]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 3s 18ms/step - loss: 0.3344 - accuracy: 0.8424 - val_loss: 0.1932 - val_accuracy: 0.9196
Epoch 2/10
192/192 [==============================] - 3s 16ms/step - loss: 0.1262 - accuracy: 0.9512 - val_loss: 0.2735 - val_accuracy: 0.9173
Epoch 3/10
192/192 [==============================] - 3s 16ms/step - loss: 0.0801 - accuracy: 0.9715 - val_loss: 0.2156 - val_accuracy: 0.9178
Epoch 4/10
192/192 [==============================] - 4s 19ms/step - loss: 0.0450 - accuracy: 0.9856 - val_loss: 0.2836 - val_accuracy: 0.9203
Epoch 5/10
192/192 [==============================] - 3s 18ms/step - loss: 0.0262 - accuracy: 0.9919 - val_loss: 0.3524 - val_accuracy: 0.9157
Epoch 6/10
192/192 [==============================] - 3s 16ms/step - loss: 0.0109 - accuracy: 0.9965 - val_loss: 0.4051 - val_accuracy: 0.9128
Epoch 7/10
192/192 [==============================] - 3s 17ms/step - loss: 0.0057 - accuracy: 0.9985 - val_loss: 0.4454 - val_accuracy: 0.9143

In [20]:
y_pred=model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [21]:
from sklearn.metrics import confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

[[3094  325]
 [ 198 2418]]
0.9133388566694284
